# Libraries

In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Real-Time Object Detection using Real-Life References Measured Object

In [36]:
# === DISPLAY FUNCTION (unchanged) ===
def display_comparison(original_img, edge_img, title1="Edge Detection", title2="Detected Object"):
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    axs[0].imshow(cv2.cvtColor(edge_img, cv2.COLOR_BGR2RGB))
    axs[0].set_title(title1)
    axs[0].axis("off")
    axs[1].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axs[1].set_title(title2)
    axs[1].axis("off")
    plt.tight_layout()
    plt.show()


# SELECT IMAGE OR VIDEO (Real Time -> Webcam=True | Detect Video -> Webcam = False)
use_webcam = False
video_path = "assets/video2.mp4"

cap = cv2.VideoCapture(0 if use_webcam else video_path)
if not cap.isOpened():
    raise IOError("Tidak bisa membuka video / webcam.")


# REAL SIZE REFERENCE (cm)
actual_width_cm = 4.0
actual_height_cm = 4.0


# === MAIN LOOP ===
while True:
    ret, img = cap.read()
    if not ret:
        break

    frame = img.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # === 1) OBJECT MASK (bright region only) ===
    _, obj_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    # === 2) LOCAL BRIGHTNESS / CONTRAST FIX (only inside detected bright object) ===
    edited = frame.astype(np.float32)
    edited += 100          # brightness +100
    edited = edited * 0.5 + 64   # reduce contrast
    edited = np.clip(edited, 0, 255).astype(np.uint8)

    edited_obj = frame.copy()
    edited_obj[obj_mask == 255] = edited[obj_mask == 255]

    # === 3) EDGE DETECTION PIPELINE (like your improved image version) ===
    gray2 = cv2.cvtColor(edited_obj, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray2, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # === 4) FIND LARGEST CONTOUR ===
    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)

        # ✅ ROTATED RECTANGLE (accurate even if object rotates)
        rect = cv2.minAreaRect(largest)
        (cx, cy), (w_rot, h_rot), angle = rect

        detected_width_px = min(w_rot, h_rot)
        detected_height_px = max(w_rot, h_rot)

        # ✅ Accurate cm-per-pixel
        cm_per_pixel_w = actual_width_cm / detected_width_px
        cm_per_pixel_h = actual_height_cm / detected_height_px
        cm_per_pixel_avg = (cm_per_pixel_w + cm_per_pixel_h) / 2

        object_width_cm = detected_width_px * cm_per_pixel_avg
        object_height_cm = detected_height_px * cm_per_pixel_avg

        # Draw bounding box
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

        cv2.putText(frame, f"Width: {object_width_cm:.2f} cm",
                    (box[0][0], box[0][1] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.putText(frame, f"Height: {object_height_cm:.2f} cm",
                    (box[0][0], box[0][1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Robust Object Measurement (Video)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_21180\2237712310.py:83: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


# Real-Time Object Measurement using ArUco

In [11]:
import cv2.aruco as aruco
import math
from typing import Optional, Tuple

import cv2, cv2.aruco as aruco
print(cv2.__version__)
print(hasattr(aruco, "detectMarkers"))

4.10.0
True


In [12]:
# Settings (change source if needed)
use_webcam = 1
video_path = "assets/video5.mp4"  # used if use_webcam == False

# Marker config (lowercase variable name as requested)
marker_size_mm = 50  # mm (50 mm -> 5.0 cm)
target_marker_id = 0
dict_choice = aruco.DICT_4X4_50

# Behavior
mode_soft = True  # if True: use last known scale when marker lost
min_contour_area = 200  # filter out tiny noise contours

In [13]:
def pixel_distance(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    """Return Euclidean distance between two 2D points."""
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

In [14]:
def detect_aruco_scale(
    gray_frame: np.ndarray,
    aruco_detector: aruco.ArucoDetector,
    target_id: int,
    marker_size_cm: float,
) -> Tuple[Optional[float], Optional[float], Optional[np.ndarray]]:
    """
    Detect target ArUco marker and compute cm-per-pixel scale.

    Args:
        gray_frame: Grayscale image.
        aruco_detector: OpenCV ArucoDetector instance.
        target_id: marker id to look for (e.g. 0).
        marker_size_cm: real-world side length of marker in centimeters.

    Returns:
        (cm_per_pixel, measured_marker_cm, corners_array)
        - cm_per_pixel: float or None if not detected
        - measured_marker_cm: measured marker side length in cm (debug), or None
        - corners_array: np.ndarray of 4x2 corner points for the found marker (or None)
    """
    corners, ids, _ = aruco_detector.detectMarkers(gray_frame)

    if ids is None or len(corners) == 0:
        return None, None, None

    ids_flat = ids.flatten()
    if target_id not in ids_flat:
        return None, None, None

    idx = int(np.where(ids_flat == target_id)[0][0])
    c = corners[idx].reshape((4, 2))

    # compute average side length in pixels (top + right) / 2
    top = pixel_distance(c[0], c[1])
    right = pixel_distance(c[1], c[2])
    marker_px = (top + right) / 2.0

    if marker_px <= 0:
        return None, None, None

    cm_per_pixel = marker_size_cm / marker_px
    measured_marker_cm = marker_px * cm_per_pixel  # should equal marker_size_cm (debug)

    return cm_per_pixel, measured_marker_cm, c

In [15]:
def detect_largest_object(gray_frame: np.ndarray) -> Optional[np.ndarray]:
    """
    Detect the largest contour (object) using a Sobel-based pipeline.
    Returns the largest contour or None if not found.
    """
    blur = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)

    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    if cv2.contourArea(largest) < min_contour_area:
        return None
    return largest

In [16]:
def measure_from_rect(rect: Tuple) -> Tuple[float, float, np.ndarray]:
    """
    Given a minAreaRect result, return (w_px, h_px, box_points)
    where w_px is the smaller side (width) and h_px the larger (height).
    """
    (cx, cy), (w_rot, h_rot), angle = rect
    w_px = min(w_rot, h_rot)
    h_px = max(w_rot, h_rot)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    return w_px, h_px, box

In [17]:
def run_measurement(
    source_is_webcam: bool = 0,
    video_path_local: str = "assets/video5.mp4",
    marker_size_mm_local: int = 50,
    marker_id_local: int = 0,
    dict_choice_local=aruco.DICT_4X4_50,
    soft_mode: bool = True,
):
    """
    Main loop: open capture, detect ArUco, detect object, measure, and display.

    Args:
        source_is_webcam: True to use webcam (0); False to use video_path_local.
        video_path_local: path to video file if not using webcam.
        marker_size_mm_local: marker physical size in millimeters.
        marker_id_local: ArUco ID to detect.
        dict_choice_local: ArUco dictionary constant.
        soft_mode: if True, reuse last known scale when marker missing.
    """
    marker_size_cm_local = marker_size_mm_local / 10.0

    # Open capture
    cap = cv2.VideoCapture(0 if source_is_webcam else video_path_local)
    if not cap.isOpened():
        raise IOError("Cannot open webcam or video file.")

    # Create ArUco detector (new API)
    aruco_dict_local = aruco.getPredefinedDictionary(dict_choice_local)
    parameters = aruco.DetectorParameters()
    aruco_detector = aruco.ArucoDetector(aruco_dict_local, parameters)

    last_cm_per_pixel_local: Optional[float] = None

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            vis = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect ArUco and compute scale
            cm_per_pixel, measured_marker_cm, marker_corners = detect_aruco_scale(
                gray, aruco_detector, marker_id_local, marker_size_cm_local
            )

            # If detected, draw blue polygon and label; else fallback
            if marker_corners is not None:
                int_box = np.int0(marker_corners)
                cv2.polylines(vis, [int_box], True, (255, 0, 0), 3)  # blue box

                # display both scale and measured marker size (option C)
                cv2.putText(vis, f"Scale: {cm_per_pixel:.6f} cm/px", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
                cv2.putText(vis, f"Marker size (meas): {measured_marker_cm:.2f} cm",
                            (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)

                last_cm_per_pixel_local = cm_per_pixel
                marker_status_text = f"Marker OK (ID {marker_id_local})"
            else:
                if last_cm_per_pixel_local is not None and soft_mode:
                    marker_status_text = "Marker lost - using last scale"
                    cv2.putText(vis, f"Scale (last): {last_cm_per_pixel_local:.6f} cm/px",
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
                else:
                    marker_status_text = "No scale - pixel units only"

                # Optionally show that no marker was found
                cv2.putText(vis, marker_status_text, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Detect largest object and measure
            largest_contour = detect_largest_object(gray)
            if largest_contour is not None:
                rect = cv2.minAreaRect(largest_contour)
                w_px, h_px, box = measure_from_rect(rect)

                # Draw object bounding box (green)
                cv2.drawContours(vis, [box], 0, (0, 255, 0), 2)

                # Use detected or last-known scale to convert to cm
                used_scale = None
                if cm_per_pixel is not None:
                    used_scale = cm_per_pixel
                elif last_cm_per_pixel_local is not None and soft_mode:
                    used_scale = last_cm_per_pixel_local

                if used_scale is not None:
                    w_cm = w_px * used_scale
                    h_cm = h_px * used_scale
                    cv2.putText(vis, f"W: {w_cm:.2f} cm", (box[0][0], box[0][1] - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    cv2.putText(vis, f"H: {h_cm:.2f} cm", (box[0][0], box[0][1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                else:
                    # show pixel fallback
                    cv2.putText(vis, f"W: {w_px:.1f} px", (box[0][0], box[0][1] - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                    cv2.putText(vis, f"H: {h_px:.1f} px", (box[0][0], box[0][1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            else:
                cv2.putText(vis, "No object contours found", (10, vis.shape[0] - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Status text near top-left (marker status)
            cv2.putText(vis, marker_status_text, (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 2)

            cv2.imshow("ArUco Measurement (function-based)", vis)

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    # run using the global settings above
    run_measurement(
        source_is_webcam=use_webcam,
        video_path_local=video_path,
        marker_size_mm_local=marker_size_mm,
        marker_id_local=target_marker_id,
        dict_choice_local=dict_choice,
        soft_mode=mode_soft,
    )

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8540\3111847449.py:50: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  int_box = np.int0(marker_corners)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_8540\606261165.py:10: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


# 🧠 Real-Time Object Measurement Using Hybrid / Robust 2.5D Measurement

This notebook demonstrates a **hybrid computer vision approach** to measure the **real-world dimensions and estimated volume** of an object in a video or live camera feed.  
The method combines **Aruco-based scale calibration** with **image-based contour analysis**, producing a robust, practical system for general object measurement and expansion tracking (e.g., food samples, mechanical parts, etc.).

---

## ⚙️ 1. Overview of the Pipeline

The system operates in several stages:

1. **Camera Feed Input**
   - The program reads from a webcam or a video file.
   - Each frame is processed in real-time.

2. **Aruco Marker Detection**
   - An ArUco marker with a known physical size (e.g., 5 cm × 5 cm) is used to compute a pixel-to-centimeter scale (`cm_per_pixel`).
   - If the marker is not visible, the last known scale is reused (`soft mode`).
   - If no marker has been detected yet, the system falls back to a default reference scale.

3. **Preprocessing**
   - The frame is converted to grayscale.
   - Brightness and contrast are adjusted locally for better object visibility.
   - This makes the system more robust under low-light or reflective conditions.

4. **Edge Detection & Contour Extraction**
   - Sobel filters compute horizontal and vertical edges.
   - Morphological closing is applied to fill gaps.
   - The largest contour is assumed to be the object of interest.

5. **Measurement & Volume Estimation**
   - The object’s width (`w`) and height (`h`) are extracted in pixels and converted to centimeters using `cm_per_pixel`.
   - A cross-sectional approximation method (based on Bai et al., 2006) estimates the volume:
     $$
     V = \sum \pi \left(\frac{D_i}{2}\right)^2 \Delta x
     $$
     where \(D_i\) is the object’s vertical diameter at each column, and \(\Delta x\) is the pixel width in cm.

6. **Error Computation**
   - The system compares the detected dimensions to known true dimensions (for calibration tests).
   - Error is expressed as a weighted RMS (root mean square) of width and height deviation:
     $$
     E = \sqrt{0.7 \cdot (\Delta W)^2 + 0.3 \cdot (\Delta H)^2}
     $$
   - This reflects physical measurement accuracy rather than pixel variability.

7. **Display and Logging**
   - Each frame displays:
     - Bounding box
     - Measured width, height, and volume
     - Margin of error (%)
     - Marker detection status
   - Measurements are continuously logged to a timestamped `.csv` file for later analysis.

---

## 🧩 2. Modular Function Breakdown

The codebase is divided into clearly defined functions for easier maintenance and testing:

| Function | Purpose |
|-----------|----------|
| **`setup_aruco_detector()`** | Initializes the ArUco detector using OpenCV 4.7+ APIs. |
| **`detect_aruco_scale()`** | Detects a specific marker ID and computes cm-per-pixel scale. |
| **`preprocess_image()`** | Enhances brightness/contrast and extracts grayscale for edge detection. |
| **`detect_largest_contour()`** | Finds the largest object contour using Sobel edge + morphological filtering. |
| **`compute_volume_and_error()`** | Computes width, height, volume, and physical error margin. |
| **`init_csv_logger()` / `log_measurement()`** | Handles structured data logging to CSV with timestamps. |
| **`run_measurement()`** | Main orchestrator: runs detection, measurement, and visualization. |

---

## 🧮 3. Volume Calculation (Bai, J., et al. (2006). "Volume and density measurement of baked products using computer vision." Inspired)

The algorithm assumes the object’s cross-section is approximately **rotationally symmetric** (e.g., buns, fruit, rounded parts).  
For each pixel column inside the contour:
- Compute the object’s vertical diameter in pixels.
- Convert to centimeters.
- Approximate the volume as the sum of circular disks:

$$
V = \sum_i \pi \left(\frac{D_i}{2}\right)^2 \Delta x
$$

This provides a **2.5D approximation** of real volume without requiring a full 3D scan.

---

## 📊 4. Margin of Error Model

The margin of error represents **real physical deviation**, not pixel noise:

$$
E = \sqrt{0.7 \cdot \left(\frac{|W_{measured} - W_{true}|}{W_{true}} \times 100\right)^2 + 0.3 \cdot \left(\frac{|H_{measured} - H_{true}|}{H_{true}} \times 100\right)^2}
$$

- Width error is given **70% weight** because it typically carries more importance for flat objects.
- Height error is given **30% weight**.
- This yields realistic percentage errors (1–5%) instead of inflated shape-based uncertainty.

This allows easy downstream analysis in Excel, MATLAB, or Python (e.g., plotting expansion over time).

---

## 🧩 6. Advantages of the Modular Design

✅ Easier maintenance — each function is independently testable  
✅ Supports **unit testing** (e.g., using `pytest` or `unittest`)  
✅ Better separation of logic → faster debugging and refactoring  
✅ Extensible — future modules can be added (e.g., camera calibration, angle correction, 3D modeling)

---

## 🧠 7. Notes on Accuracy

To achieve sub-2% measurement accuracy:
- Keep the **camera fixed** (no tilt between sessions)
- Use **Aruco marker** at the same depth plane as the object
- Ensure **uniform lighting** (avoid specular reflections)
- Calibrate `marker_size_cm_local` precisely with a ruler

---

## ✅ 8. Run the System

Run the measurement pipeline using:

```python
if __name__ == "__main__":
    run_measurement(use_webcam=True)


In [27]:
import csv
from datetime import datetime

In [28]:
def setup_aruco_detector():
    """Initialize ArUco detector (OpenCV ≥4.7)."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    aruco_params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)
    return detector

In [29]:
def detect_aruco_scale(gray, detector, marker_id, marker_size_cm):
    """Detect ArUco marker and compute scale (cm/pixel)."""
    corners, ids, _ = detector.detectMarkers(gray)
    if ids is not None and marker_id in ids:
        idx = np.where(ids == marker_id)[0][0]
        c = corners[idx][0]
        side_px = np.mean([
            np.linalg.norm(c[0]-c[1]),
            np.linalg.norm(c[1]-c[2]),
            np.linalg.norm(c[2]-c[3]),
            np.linalg.norm(c[3]-c[0])
        ])
        cm_per_pixel = marker_size_cm / side_px
        measured_marker_cm = side_px * cm_per_pixel
        return cm_per_pixel, measured_marker_cm, c
    return None, None, None

In [30]:
def preprocess_image(frame):
    """Preprocess image for edge-based contour detection."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, obj_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    edited = frame.astype(np.float32)
    edited += 100
    edited = edited * 0.5 + 64
    edited = np.clip(edited, 0, 255).astype(np.uint8)

    edited_obj = frame.copy()
    edited_obj[obj_mask == 255] = edited[obj_mask == 255]
    return cv2.cvtColor(edited_obj, cv2.COLOR_BGR2GRAY)

In [31]:
def detect_largest_contour(gray2):
    """Detect and return the largest contour."""
    blur = cv2.GaussianBlur(gray2, (9, 9), 0)
    sobel_x = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((7, 7), np.uint8)
    thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    return max(contours, key=cv2.contourArea)

In [32]:
def compute_volume_and_error(contour, cm_per_pixel, true_dims=(17.5, 6.5)):
    """Compute width, height, volume, and margin of error."""
    x, y, w, h = cv2.boundingRect(contour)
    mask = np.zeros_like(cv2.cvtColor(np.zeros((h, w), np.uint8), cv2.COLOR_GRAY2BGR))
    bun_mask = np.zeros((h, w), np.uint8)
    cv2.drawContours(bun_mask, [contour], -1, 255, -1)

    diameters_px = []
    for col in range(x, x + w):
        col_pix = np.where(bun_mask[:, col - x] > 0)[0]
        if len(col_pix) > 1:
            diameters_px.append(col_pix.max() - col_pix.min())

    if len(diameters_px) <= 10:
        return None

    diameters_px = np.array(diameters_px)
    diameters_cm = diameters_px * cm_per_pixel
    delta_x_cm = cm_per_pixel
    volume_cm3 = np.sum(np.pi * (diameters_cm / 2) ** 2 * delta_x_cm)

    # === Width/Height in cm ===
    w_cm = w * cm_per_pixel
    h_cm = h * cm_per_pixel

    # === Error estimation (physical comparison only) ===
    true_w, true_h = true_dims
    width_diff_pct  = abs((w_cm - true_w) / true_w) * 10
    height_diff_pct = abs((h_cm - true_h) / true_h) * 10
    err_total = np.sqrt(0.7 * (width_diff_pct**2) + 0.3 * (height_diff_pct**2))

    return w_cm, h_cm, volume_cm3, err_total, (x, y, w, h)

In [33]:
def init_csv_logger():
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"object_measurements_{timestamp}.csv"
    with open(filename, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Frame Index", "Marker Status", "cm/pixel",
                         "Width (cm)", "Height (cm)", "Volume (cm^3)", "Margin of Error (%)"])
    return filename


def log_measurement(filename, frame_idx, marker_status, cm_per_pixel, w_cm, h_cm, volume_cm3, err_total):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, mode="a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, frame_idx, marker_status,
                         round(cm_per_pixel, 6), round(w_cm, 3), round(h_cm, 3),
                         round(volume_cm3, 3), round(err_total, 3)])

In [34]:
def run_measurement(use_webcam=True, video_path="assets/video2.mp4"):
    cap = cv2.VideoCapture(0 if use_webcam else video_path)
    if not cap.isOpened():
        raise IOError("Cannot open video / webcam.")

    # Configuration
    reference_width_cm, reference_height_cm = 4.0, 2.1
    marker_id, marker_size_cm = 0, 5.1
    soft_mode = True
    last_cm_per_pixel = None
    marker_status = "No marker detected"

    aruco_detector = setup_aruco_detector()
    csv_filename = init_csv_logger()
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        vis = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # --- Detect ArUco marker ---
        cm_per_pixel, measured_marker_cm, marker_corners = detect_aruco_scale(
            gray, aruco_detector, marker_id, marker_size_cm
        )

        if marker_corners is not None:
            int_box = np.intp(marker_corners)
            cv2.polylines(vis, [int_box], True, (255, 0, 0), 2)
            cv2.putText(vis, f"Scale: {cm_per_pixel:.6f} cm/px", (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 0), 2)
            last_cm_per_pixel = cm_per_pixel
            marker_status = f"Marker OK (ID {marker_id})"
        else:
            if last_cm_per_pixel is not None and soft_mode:
                cm_per_pixel = last_cm_per_pixel
                marker_status = f"Marker lost - using last scale = {last_cm_per_pixel:.6f} cm/px"
            else:
                marker_status = "No marker - using reference tin size"
                h_img, w_img = gray.shape
                cm_per_pixel_x = reference_width_cm / w_img
                cm_per_pixel_y = reference_height_cm / h_img
                cm_per_pixel = (cm_per_pixel_x + cm_per_pixel_y) / 2

        # --- Object detection ---
        gray2 = preprocess_image(frame)
        largest_contour = detect_largest_contour(gray2)

        if largest_contour is not None:
            result = compute_volume_and_error(largest_contour, cm_per_pixel)
            if result:
                w_cm, h_cm, volume_cm3, err_total, (x, y, w, h) = result

                cv2.rectangle(vis, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(vis, f"Width: {w_cm:.2f} cm  Height: {h_cm:.2f} cm", (x, y - 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(vis, f"Vol: {volume_cm3:.1f} cm^3 (+/- {err_total:.1f}%)",
                            (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                log_measurement(csv_filename, frame_idx, marker_status, cm_per_pixel, w_cm, h_cm, volume_cm3, err_total)

        cv2.putText(vis, marker_status, (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 2)

        cv2.imshow("Object Volume Measurement (Modular)", vis)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"\n✅ Data saved to: {csv_filename}")

In [35]:
if __name__ == "__main__":
    run_measurement(use_webcam=True)


✅ Data saved to: object_measurements_2025-11-13_19-52-19.csv
